# phase 1 分完重點後，切換到對應的 thread

In [ ]:
import os
import time
from openai import OpenAI
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')
print(OpenAI.api_key)

In [ ]:
import pandas as pd
import numpy as np

csv_file = './data/embeddingsV3_3072.csv'
df = pd.read_csv(csv_file)
data = df.to_numpy()

In [ ]:
import faiss

index = faiss.IndexFlatL2(data.shape[1])
index.add(data)

In [ ]:
def get_similar_qa(input):
    response = client.embeddings.create(input=input, model="text-embedding-3-large", dimensions=3072)
    question = np.array(response.data[0].embedding, ndmin=2)
    k = 3
    distances, indices = index.search(question, k)
    csv_question = 'data\manual.csv'
    qdf = pd.read_csv(csv_question)
    qarray = qdf.to_numpy()
    answer = [(qarray[i], float(dist)) for dist, i in zip(distances[0], indices[0])]
    history = ""
    min_distance = 1
    Qcount = 0
    for i in range(0,k):
        if answer[i][1]<min_distance:
            Qcount+=1
            history = history +str(i+1)+". "+ answer[i][0][0]+"\n"+answer[i][0][1]+"\n"
    return Qcount, history

def get_similar_flow(input):
    pass

In [ ]:
instructions = [
    "You are Flora customer service chat bot. You are prohibited replying unrelated to Flora. ",
    "When user ask a question, You must reply based on 3 history questions and answers. ",
    "If 3 history questions and answers not related to my question, you must reply only a special mark:'*'. ",
    "If history answer have flow of steps inside, you must follow the flow, resolving the problem one step at a time. ",
    "If the user asks another question before the previous question is solved, focus dealing with the previous question and tell them to open another feedback request to solve thier other questions. ",
    "When you encounter user providing email, you must call check_email for verification. "
]

instruction = ""
for i in instructions:
    instruction += i

print(instruction)

In [ ]:
assistant = client.beta.assistants.create(
    name = "Flora's Customer Service",
    instructions = instruction,
    model = "gpt-3.5-turbo-1106",
    tools = [
        {
            "type": "function",
            "function": {
                "name": "check_email",
                "description": "checking email is exist in our database or not.True represent exists and you can proceed next step.False represent not exist and you have to try asking user again",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "email": {
                            "type": "string",
                            "description": "user's email"
                        }
                    },
                    "required": ["email"]
                }
            }
        }
    ]
)

thread = client.beta.threads.create()

In [ ]:
init = 1

key_point = ""

# Chat start from here

一開始用 key point 給回應(init = 1)，後續是和 user 溝通 (init = 0)

In [ ]:
follow_up_conversation = ""

In [ ]:
user_prompt = ""

if init == 1:
    user_input = key_point
    similar_qa = get_similar_qa(user_input)
    similar_flow = get_similar_flow(user_input)

    user_prompt += similar_flow

    if similar_qa[0] > 0:
        user_prompt += "\nHere is " + str(similar_qa[0]) + " history questions and answers related to this question according to Flora's database:\n" + similar_qa[1] + "\n"

    user_prompt = user_prompt + "Here is the user input: " + user_input
elif init == 0:
    user_input = follow_up_conversation
    user_prompt = user_input

init = 0
print(user_prompt)

In [ ]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = user_prompt
)

In [ ]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id
)

while run.status == "queued" or run.status == "in_progress":
    run = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id
    )
print(run.status)

In [ ]:
if run.status == "requires_action":
    if run.required_action.submit_tool_outputs.tool_calls[0].function.name == "check_email":
        print("function calling: check_email")
        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=[
                {
                    "tool_call_id": run.required_action.submit_tool_outputs.tool_calls[0].id,
                    "output": "True"
                }
            ]
        )
else:
    print("No function calling")

In [ ]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id,
    order = 'asc'
)

for i in messages.data[-10:]:
    print(i.content[0].text.value)
    print("==========")

# Delete Assistant

In [ ]:
client.beta.assistants.delete(assistant.id)

In [ ]:
client.beta.assistants.list(
    order="desc",
    limit="20",
)